## ffmpeg drawtext
- 이미지 혹은 영상 특정 위치에 고정된 글자를 삽입

In [1]:
import os

def text(mp4_file, text, output_mp4_path, x="center"):
    if x == "center":
        w = "(w-text_w)/2"
        h = "(h-text_h)/2"
    elif x == "left":
        w = "10"
        h = "h - 30"
    else:
        w = "0"
        y = "0"
    os.system(f"""ffmpeg -i {mp4_file}.mp4 -vf drawtext="text={text}: \
    fontcolor=white: fontsize=24: box=1: boxcolor=black@0.5: \
    boxborderw=5: x={w}: y={h}" -codec:a copy {output_mp4_path}.mp4""")

    
def text_bottom_left(mp4_file, text, output_mp4_path, fontsize):
    os.system(f"""ffmpeg -i {mp4_file}.mp4 -vf drawtext="font=NanumBarunGothic: basetime=1: \
    text={text}: fontcolor=white: fontsize={fontsize}: box=1: boxcolor=black@0.5: \
    boxborderw=5: x=(w-text_w)/10*8: y=(h-text_h)/4*3" -codec:a copy {output_mp4_path}.mp4 -y""")

In [ ]:
# 테스트용 영상 3초짜리로 잘라 만들기
os.system(f"""ffmpeg -i {input_mp4_path}.mp4 -acodec copy -vcodec copy -ss 0 -t 3 {output_mp4_path}.mp4""")

In [43]:
mp4_file = '../sound_files/충무로_3s'
text_bottom_left(mp4_file, "한국어도 되나???",50)

## 동영상에 SRT자막 파일 만들어 입히기
- 타임라인과 자막이 함께 들어가는 파일, 확장자 srt

In [2]:
import datetime
import pandas as pd

### 예시로 보여주기 위해 타임라인을 임의로 만들기

In [8]:
pd.timedelta_range(start = datetime.timedelta(seconds=0.0),end=datetime.timedelta(seconds=60.0),freq='0.1S')

TimedeltaIndex([       '0 days 00:00:00', '0 days 00:00:00.100000',
                '0 days 00:00:00.200000', '0 days 00:00:00.300000',
                '0 days 00:00:00.400000', '0 days 00:00:00.500000',
                '0 days 00:00:00.600000', '0 days 00:00:00.700000',
                '0 days 00:00:00.800000', '0 days 00:00:00.900000',
                ...
                '0 days 00:00:59.100000', '0 days 00:00:59.200000',
                '0 days 00:00:59.300000', '0 days 00:00:59.400000',
                '0 days 00:00:59.500000', '0 days 00:00:59.600000',
                '0 days 00:00:59.700000', '0 days 00:00:59.800000',
                '0 days 00:00:59.900000',        '0 days 00:01:00'],
               dtype='timedelta64[ns]', length=601, freq='100L')

In [9]:
one_m_ls = pd.timedelta_range(start = datetime.timedelta(seconds=0.0),end=datetime.timedelta(seconds=60.0),freq='0.1S')
one_m_ls = [str(n)[7:] for n in one_m_ls]

In [10]:
time_df = pd.DataFrame(one_m_ls, columns=['start_time'])
time_df['end_time']= time_df.start_time.shift(-1).fillna(0)
time_df = time_df.iloc[:-1]
time_df['start_time'] = [t+':000000' if len(t)==8 else t for t in time_df['start_time']]
time_df['end_time'] = [t+':000000' if len(t)==8 else t for t in time_df['end_time']]
time_df["seq_num"] = time_df.index.astype(int) + 1
time_df.seq_num = time_df.seq_num.astype('str')
time_df

,start_time,end_time,seq_num
0,00:00:00:000000,00:00:00.100000,1
1,00:00:00.100000,00:00:00.200000,2
2,00:00:00.200000,00:00:00.300000,3
3,00:00:00.300000,00:00:00.400000,4
4,00:00:00.400000,00:00:00.500000,5
...,...,...,...
595,00:00:59.500000,00:00:59.600000,596
596,00:00:59.600000,00:00:59.700000,597
597,00:00:59.700000,00:00:59.800000,598
598,00:00:59.800000,00:00:59.900000,599


In [11]:
# make start string
time_df['srt_time'] = time_df['start_time'].str[:8] + "," +  time_df['start_time'].str[9:12]+ " --> "
# make stop string
time_df['srt_time'] = time_df['srt_time'] + time_df['end_time'].str[:8] + "," + time_df['end_time'].str[9:12]
time_df['subtitle'] = [f"sub{i+1}" for i in range(len(time_df))]
time_df["srt"] = time_df['seq_num'].str[:] + "\n" \
                    + time_df['srt_time'].str[:] + "\n" \
                    + time_df['subtitle'] \
                    + "\n\n"
time_df

,start_time,end_time,seq_num,srt_time,subtitle,srt
0,00:00:00:000000,00:00:00.100000,1,"00:00:00,000 --> 00:00:00,100",sub1,"1\n00:00:00,000 --> 00:00:00,100\nsub1\n\n"
1,00:00:00.100000,00:00:00.200000,2,"00:00:00,100 --> 00:00:00,200",sub2,"2\n00:00:00,100 --> 00:00:00,200\nsub2\n\n"
2,00:00:00.200000,00:00:00.300000,3,"00:00:00,200 --> 00:00:00,300",sub3,"3\n00:00:00,200 --> 00:00:00,300\nsub3\n\n"
3,00:00:00.300000,00:00:00.400000,4,"00:00:00,300 --> 00:00:00,400",sub4,"4\n00:00:00,300 --> 00:00:00,400\nsub4\n\n"
4,00:00:00.400000,00:00:00.500000,5,"00:00:00,400 --> 00:00:00,500",sub5,"5\n00:00:00,400 --> 00:00:00,500\nsub5\n\n"
...,...,...,...,...,...,...
595,00:00:59.500000,00:00:59.600000,596,"00:00:59,500 --> 00:00:59,600",sub596,"596\n00:00:59,500 --> 00:00:59,600\nsub596\n\n"
596,00:00:59.600000,00:00:59.700000,597,"00:00:59,600 --> 00:00:59,700",sub597,"597\n00:00:59,600 --> 00:00:59,700\nsub597\n\n"
597,00:00:59.700000,00:00:59.800000,598,"00:00:59,700 --> 00:00:59,800",sub598,"598\n00:00:59,700 --> 00:00:59,800\nsub598\n\n"
598,00:00:59.800000,00:00:59.900000,599,"00:00:59,800 --> 00:00:59,900",sub599,"599\n00:00:59,800 --> 00:00:59,900\nsub599\n\n"


In [121]:
srt = time_df.srt.copy()
srt.to_csv('subtitle.srt', header=None, mode='w', index=False)

In [122]:
# srt자막에서 불필요한 쌍따옴표 제거하기

# Read in the file
with open('subtitle.srt', 'r') as file :
    filedata = file.read()

# Replace the target string
filedata = filedata.replace('"', '')

# Write the file out again
with open('subtitle.srt', 'w') as file:
    file.write(filedata)
  
!head subtitle.srt

1
00:00:00,000 --> 00:00:00,100
sub1


2
00:00:00,100 --> 00:00:00,200
sub2




In [123]:
# 동영상에 srt자막파일 입히기
os.system(f"""ffmpeg -i ../sound_files/충무로_1m.mp4 -vf "subtitles=subtitle.srt" -c:a aac ../sound_files/충무로_1m_sub.mp4 -y""")

0

### 시연용 영상 추출하기
- -ss: 시작시점(초), -t: 영상길이(초) 

In [12]:
os.system(f"""ffmpeg -i ../sound_files/mp4_sub/충무로_1024_voice.mp4 -ss 0 -t 15 ../sound_files/mp4_sub/충무로_목소리분류_1.mp4 -y""")
os.system(f"""ffmpeg -i ../sound_files/mp4_sub/충무로_1024_voice.mp4 -ss 117 -t 20 ../sound_files/mp4_sub/충무로_목소리분류_2.mp4 -y""")

0

In [11]:
os.system(f"""ffmpeg -i ../sound_files/mp4_sub/충무로_512_gender.mp4 -ss 628 -t 15 ../sound_files/mp4_sub/충무로_성별분류_1.mp4 -y""")
os.system(f"""ffmpeg -i ../sound_files/mp4_sub/충무로_512_gender.mp4 -ss 1098 -t 18 ../sound_files/mp4_sub/충무로_성별분류_2.mp4 -y""")

0